In [46]:
from xmlrpc.client import DateTime

import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from yahooquery import Ticker
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import seaborn as sns
from datetime import datetime

## importando pelo yahoo finance

# Todos os filtros para buscar reits a partir do YahooQuery

## Fundos imobiliarios

In [3]:
pd_fiis = pd.read_csv('statusinvest-busca-avancada.csv', sep=';')
len(pd_fiis['TICKER'])


579

## REITs

In [3]:
#s = yq.Screener()
#
#lista_screeners = ['reit_diversified',
#    'reit_healthcare_facilities',
#    'reit_hotel_motel',
#    'reit_industrial',
#    'reit_office',
#   'reit_residential',
#   'reit_retail']
#
#lista_cod_reits = []
#reits = s.get_screeners(lista_screeners, 100)
#pd_reits = pd.DataFrame(reits)
#json_reit_diversified = pd_reits.iloc[9]

#for reit in json_reit_diversified:
#    for item in reit:
#        lista_cod_reits.append(item['symbol'])

#len(lista_cod_reits)

# Realizando a requisição dos REITs

In [53]:
dataJson = list()

for Fii in pd_fiis['TICKER']:
    dataJson.append(Ticker(Fii + '.SA'))
len(dataJson)

579

In [63]:
type(dataJson[2].all_financial_data())
#start_date = balance_sheet['asOfDate'].iloc[0]
#start_date

pandas.core.frame.DataFrame

In [ ]:
all_stock_complete = []

for data_stocks in dataJson:
    try:
        balance_sheet = data_stocks.all_financial_data()

        if isinstance(balance_sheet, str):
            continue  # Pula para a próxima iteração se for uma string

        # Obtendo as datas inicial e final
        start_date, end_date = balance_sheet['asOfDate'].iloc[[0, -1]]

        # Obtendo o histórico de preços no intervalo desejado
        historic_value = data_stocks.history(start=start_date, end=end_date)

        # Limpando e preparando os DataFrames
        balance_sheet = (
            balance_sheet
            .fillna(0)  # Substitui NaNs por 0
            .rename(columns={'asOfDate': 'date'})  # Renomeia a coluna
            .assign(date=lambda df: pd.to_datetime(df['date']),  # Converte para datetime
                    year=lambda df: df['date'].dt.year)  # Extrai o ano
        )

        historic_value = (
            historic_value
            .reset_index()  # Transforma índices em colunas para evitar problemas
            .assign(date=lambda df: pd.to_datetime(df['date']),  # Converte para datetime
                    year=lambda df: df['date'].dt.year)  # Extrai o ano
        )

        # Juntando os DataFrames com base no ano
        complete_values = pd.merge(balance_sheet, historic_value, on='year', how='inner')

        # Ajustando o DataFrame final
        complete_values.drop(columns=['date_x'], inplace=True)  # Remove data duplicada
        complete_values.rename(columns={'date_y': 'date'}, inplace=True)

        # Adicionando a coluna 'symbol'
        complete_values['symbol'] = historic_value['symbol']

        # Armazena o resultado
        all_stock_complete.append(complete_values)
    except:
        print(data_stocks)

In [ ]:
len(all_stock_complete)

In [71]:
df_stocks = pd.concat(all_stock_complete).drop_duplicates().reset_index(drop=True)
df_stocks = df_stocks.fillna(0)
df_stocks.to_excel('all_stocks.xlsx', index=False)

In [31]:
all_stocks = pd.read_csv('all_stocks.csv')

# Filtrar todos os fundos que o request na API conseguiu ter acesso aos dados:

### Exemplo de exibição, simpleInfo é uma lista de dataframes

## Filtrar com base na media de volume - Minimo: 1000

In [32]:
all_stocks = all_stocks.drop(['currencyCode', 'periodType'], axis=1)

In [33]:
info_target = all_stocks[['open', 'high', 'low', 'close', 'symbol', 'date']]
all_stocks = all_stocks.drop(['open', 'high', 'low', 'close', 'symbol', 'date'], axis=1)
info_target

,open,high,low,close,symbol,date
0,74.480003,74.480003,73.820000,74.370003,ABCP11.SA,2021-01-04
1,74.370003,74.519997,73.849998,74.430000,ABCP11.SA,2021-01-05
2,74.419998,74.419998,73.809998,73.959999,ABCP11.SA,2021-01-06
3,73.949997,74.379997,73.949997,74.040001,ABCP11.SA,2021-01-07
4,74.050003,74.389999,74.050003,74.269997,ABCP11.SA,2021-01-08
...,...,...,...,...,...,...
124746,800.000000,810.000000,800.000000,810.000000,ZIFI11.SA,2023-12-19
124747,810.000000,810.000000,762.010010,810.000000,ZIFI11.SA,2023-12-20
124748,810.000000,810.000000,810.000000,810.000000,ZIFI11.SA,2023-12-22
124749,849.000000,849.000000,849.000000,849.000000,ZIFI11.SA,2023-12-26


In [34]:
info_target['close']

0          74.370003
1          74.430000
2          73.959999
3          74.040001
4          74.269997
             ...    
124746    810.000000
124747    810.000000
124748    810.000000
124749    849.000000
124750    849.000000
Name: close, Length: 124751, dtype: float64

## Adicionar colunas com a regressão linear, comparando com o valor: b3, IPCA, tesouro direto

In [20]:
ibova = pd.DataFrame(yf.Ticker('BOVA11.SA').history(period="max"))
ibova.index = ibova.index.strftime('%d/%m/%Y')
ibova

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
17/04/2009,45.700001,45.980000,45.599998,45.849998,274700,0.0,0.0
20/04/2009,44.790001,44.900002,44.070000,44.419998,265800,0.0,0.0
22/04/2009,44.610001,45.240002,44.509998,44.990002,234800,0.0,0.0
23/04/2009,45.189999,45.900002,44.860001,45.900002,305900,0.0,0.0
24/04/2009,46.169998,46.849998,46.099998,46.689999,325600,0.0,0.0
...,...,...,...,...,...,...,...
31/01/2025,123.959999,124.400002,122.930000,123.000000,8503720,0.0,0.0
03/02/2025,122.889999,123.379997,122.480003,122.639999,3562259,0.0,0.0
04/02/2025,122.430000,122.760002,121.580002,121.910004,2660619,0.0,0.0


In [44]:
dados1 = np.array(all_stocks)
dados2 = np.array(info_target['close'])

X_train, X_test, y_train, y_test = train_test_split(dados1, dados2, test_size=0.2, random_state=42)
# Criar e treinar o modelo
# Criando o modelo Random Forest
modelo = RandomForestRegressor(n_estimators=65, max_depth=5, random_state=42)

# Treinando o modelo
modelo.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = modelo.predict(X_test)

# Avaliação do modelo
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R² Score: {r2:.4f}")

Mean Absolute Error (MAE): 13.2381
Mean Squared Error (MSE): 791.2360
R² Score: 1.0000
R² Treino: 1.0000
R² Teste: 1.0000


In [42]:
y_train_pred = modelo.predict(X_train)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_pred)

print(f"R² Treino: {r2_train:.4f}")
print(f"R² Teste: {r2_test:.4f}")


R² Treino: 1.0000
R² Teste: 1.0000
